In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class
from RP_train_all_at_once import train_end_to_end_RP_combined
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
from RP_data_loader import Custom_RP_Dataset
from sklearn.model_selection import LeaveOneGroupOut
from torch.utils.data import DataLoader, TensorDataset



In [5]:
torch.cuda.empty_cache()

In [2]:
def train_different_classes(RP_train_gen, RP_val_gen, train_set, test_set, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        # print(i)
        acc, balanced_acc = train_end_to_end_RP_combined(RP_train_gen, RP_val_gen, train_set, test_set, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

In [3]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")

datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Custom_RP_Dataset(path=data_file, total_points=2000, tpos=120, tneg=300, windowSize=3, sfreq=1000))
f.close()


training_set = torch.utils.data.ConcatDataset(datasets_list)

data_len = len(training_set)
print("dataset len is", len(training_set))

train_len = int(data_len*0.8)
val_len = data_len - train_len

training_set, validation_set = torch.utils.data.random_split(training_set, [train_len, val_len])

print("one dataset is", len(datasets_list[0]))

params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 4}
max_epochs = 40
training_generator = torch.utils.data.DataLoader(training_set, **params)
validation_generator = torch.utils.data.DataLoader(validation_set, **params)

print("len of the dataloader is:",len(training_generator))

# cuda setup if allowed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0


Loading Data
Processing MouseCKA1_030515_HCOFTS
1914000
(58, 3000, 11)
(2000,)
Processing MouseCKA1_030615_HCOFTS
1881000
(57, 3000, 11)
(2000,)
Processing MouseCKL1_062514_HCOFTS
2013000
(61, 3000, 11)
(2000,)
Processing MouseCKB9_022715_HCOFTS
1980000
(60, 3000, 11)
(2000,)
Processing MouseCKB9_022815_HCOFTS
2079000
(63, 3000, 11)
(2000,)
Processing MouseCKL7_063014_HCOFTS
1914000
(58, 3000, 11)
(2000,)
Processing MouseCKL5_063014_HCOFTS
1881000
(57, 3000, 11)
(2000,)
Processing MouseCKL5_070114_HCOFTS
1848000
(56, 3000, 11)
(2000,)
Processing MouseCKL7_070114_HCOFTS
1848000
(56, 3000, 11)
(2000,)
Processing MouseCKN1_063014_HCOFTS
1848000
(56, 3000, 11)
(2000,)
Processing MouseCKN2_070214_HCOFTS
1815000
(55, 3000, 11)
(2000,)
Processing MouseCKN3_070214_HCOFTS
1815000
(55, 3000, 11)
(2000,)
Processing MouseCKN3_070314_HCOFTS
1848000
(56, 3000, 11)
(2000,)
Processing MouseCKO1_070214_HCOFTS
1815000
(55, 3000, 11)
(2000,)
Processing MouseCKN2_070314_HCOFTS
1815000
(55, 3000, 11)
(2000

In [4]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
    
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


logo = LeaveOneGroupOut()
logo.get_n_splits(x_vals, y_vals, groups)

result_dict = {}
# dtype=torch.int32
for train_index, test_index in logo.split(x_vals, y_vals, groups):
    group_num = groups[test_index][0]
    print("Leaving out mouse number:", group_num)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    # def train_different_classes(RP_train_gen, RP_val_gen, train_set, test_set, epochs, sample_list):

    accuracy, balanced_accuracy = train_different_classes(training_generator, validation_generator, training_set, test_set, 50, num_samples)
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)



Loading Data
Processing MouseCKA1_030515_HCOFTS
removed 0 unknown entries
Processing MouseCKA1_030615_HCOFTS
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022815_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_070114_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_070114_HCOFTS
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070314_HCOFTS
removed 0 unknown entries
Processing 

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 10.76 GiB total capacity; 9.85 GiB already allocated; 0 bytes free; 9.87 GiB reserved in total by PyTorch)

In [5]:
smallest_class = smallest_class_len(training_set)
num_samples=[]
temp=1
while temp < smallest_class:
    num_samples.append(temp)
    temp*=10
num_samples.append(None)
print(num_samples)

[1, 10, 100, 1000, None]


In [7]:
num_samples=[1000, None]
RP_2loss_vals = train_different_classes(training_generator, training_generator, validation_set, 100, num_samples)

downstream batches: 20
pretext batches: 250
Start Training Full
[0.6312960554833117]
downstream batches: 68
pretext batches: 250
Start Training Full
[0.6312960554833117, 0.693194625054183]


In [8]:
print(RP_vals)

[0.3046380580840919, 0.4250541829215431, 0.5237104464672735, 0.59913307325531, 0.6680537494581708]
